In [65]:
import subprocess as sc
import time
import re
import requests

In [83]:
root_endpoint = "http://localhost:8000/"
auth_endpoint = root_endpoint + 'api/auth/'

In [89]:
def get_logs(tail='50', svc=''):
    cmd = ['docker-compose', 'logs', '--no-color', f'--tail={tail}']
    if svc:
        cmd.append('--no-log-prefix')
        cmd.append(svc)
    return sc.run(cmd, capture_output=True)

In [43]:
logs = get_logs('desk2-api')
re.findall('Starting development server at http://0.0.0.0:8000/', logs.stdout.decode())

['Starting development server at http://0.0.0.0:8000/']

In [96]:
def poll_logs_until_match(pattern, tail='50', svc='', timeout=30):
    while timeout > 0:
        logs = get_logs(tail=tail, svc=svc).stdout.decode()
        matches = re.findall(pattern, logs)
        if matches:
            return matches
        print(f"couldn't get a match for pattern '{pattern}'; sleeping...")
        time.sleep(5)
        timeout -= 1
    return []       

In [56]:
poll_logs_until_match('Starting', 'desk2-api')

['Starting']

In [108]:
def stop_services():
    print('stopping services and exiting...')
    sc.run(['docker-compose', 'down'])

def start_services():
    print("running 'docker-compose up -d postgres'...")
    sc.run(['docker-compose', 'up', '-d', 'postgres'], capture_output=True)
    print("running 'docker-compose up -d redis'...")
    sc.run(['docker-compose', 'up', '-d', 'redis'], capture_output=True)
    postgres_ready = poll_logs_until_match('database system is ready to accept connections', svc='postgres')
    if not postgres_ready:
        print("couldn't start postgres")
        stop_services()
        return False
    
    print("running 'docker-compose up -d desk2-api'...")
    sc.run(['docker-compose', 'up', '-d', 'desk2-api'], capture_output=True)
    
    res = poll_logs_until_match('Starting development server', svc='desk2-api')
    if not res:
        print("ERROR: couldn't start services")
        stop_services()
        return False
    
    unapplied_migrations = poll_logs_until_match('unapplied migration', tail='all', svc='desk2-api', timeout=1)
    if unapplied_migrations:
        print("ERROR: couldn't apply migrations")
        stop_services()
        success = False
        
    print('started services')

In [114]:
start_services()

running 'docker-compose up -d postgres'...
running 'docker-compose up -d redis'...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
couldn't get a match for pattern 'database system is ready to accept connections'; sleeping...
running 'docker-compose up -d desk2-api'...
couldn't get a match for pattern 'Starting development server'; sleeping...
ERROR: couldn't apply migrations
stopping services and exiting...


Stopping tests_desk2-api_1 ... 
Stopping tests_redis_1     ... 
Stopping tests_postgres_1  ... 
Stopping tests_desk2-api_1 ... done
Stopping tests_redis_1     ... done
Stopping tests_postgres_1  ... done
Removing tests_desk2-api_1 ... 
Removing tests_redis_1     ... 
Removing tests_postgres_1  ... 
Removing tests_redis_1     ... done
Removing tests_postgres_1  ... done
Removing tests_desk2-api_1 ... done
Removing network tests_default


started services


In [84]:
requests.get(auth_endpoint + 'user/')

<Response [401]>

In [80]:
user_registration_obj = {
    "first_name": "name",
    "last_name": "last_name",
    "middle_name": "middle_name",
    "email": "randomemail@gmail.com",
    "password": "opa_Wasm1112233",
    "profile_type": "student",
    "student_card_id": "12345678"
}
response = requests.post(auth_endpoint + 'user/', data=user_registration_obj)

In [82]:
response.json()

{'email-token': 'This field is required.'}

In [111]:
response = requests.post(auth_endpoint + 'token/send-token/email-confirm/', data={"email": "randomemail@gmail.com"})

In [112]:
response

<Response [200]>